In [44]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
from scipy.ndimage.filters import correlate 
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
import math

In [45]:
def load_image(path, mode=cv2.IMREAD_GRAYSCALE):
    '''
    :param path: image path
    :param mode: load mode, cv2.IMREAD_GRAYSCALE, cv2.IMREAD_COLOR
    :return: loaded img
    '''
    img = cv2.imread(path,mode)
    return img


In [46]:

def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)


In [57]:
def generated_hybrid_image(image1, image2, kernel_size1, sigma1, kernel_size2, sigma2):
    high_pass_filter = gaussian_filter(np.zeros((kernel_size1,kernel_size1))
                                       , sigma=sigma1)
    high_pass = image1 - correlate(image1, high_pass_filter)
    
    low_pass_filter = gaussian_filter(np.zeros((kernel_size2,kernel_size2))
                                      , sigma=sigma2)
    low_pass = correlate(image2, low_pass_filter)
    
    hybrid_img = (low_pass + high_pass)
    return hybrid_img

In [58]:
def generate_and_show_image(img_path1, img_path2, kernel_size1, sigma1, kernelsize_2,
                            sigma2, saved_name):
    '''
    :param img_path1: str, path to img1
    :param img_path2: str, path to img2
    :param kernel_size1: kernel size for img1
    :param sigma1: sigma for gaussian kernel1
    :param kernelsize_2: kernel size for img2
    :param sigma2: sigma for gaussian kernel2
    :param saved_name: str, name for the saved image
    :return: none, already show and saved the image
    '''
    high_frequency = load_image(PATH_C_1)
    low_frequency = load_image(PATH_C_2)
    high_frequency = high_frequency.astype(np.single)/255 
    low_frequency = low_frequency.astype(np.single)/255
    # print(high_frequency.shape)
    # print(low_frequency.shape)
    
    hybrid_img_C = generated_hybrid_image(high_frequency, low_frequency, kernel_size1, sigma1, kernel_size2, sigma2)
    
    ''' Visualize and save outputs '''
    plt.figure(1)
    plt.imshow(normalize(low_frequency), cmap='gray')
    plt.figure(2)
    plt.imshow(normalize(high_frequency+0.5), cmap='gray')
    plt.figure(3)
    plt.imshow(normalize(hybrid_img_C), cmap='gray')
    plt.show()
    # print(hybrid_img_C.dtype)
    cv2.imwrite('hybrid_c.png', 255 * normalize(hybrid_img_C)) 
   


In [59]:
# test for sample image
    
PATH_C_1 = "data\\part1\\c1.jpg"
PATH_C_2 = "data\\part1\\c2.jpg"
sigma1 = 25
kernel_size1 = 5
sigma2 = 10
kernel_size2 = 5

generate_and_show_image(PATH_C_1, PATH_C_2, kernel_size1,sigma1,
                        kernel_size2,sigma2,"hybrid_c.png")
    